In [ ]:
import os
import glob
import pandas as pd

def load_pm_pollutants_fixed(dir_path="data/kaggle_csvs", out_file="data/kaggle_pm_merged_fixed.csv"):
    files = glob.glob(os.path.join(dir_path, "*.csv"))
    print(f"Found {len(files)} CSV files.")
    os.makedirs(os.path.dirname(out_file), exist_ok=True)

    first = True
    for i, f in enumerate(files, 1):
        try:
            # Read column headers first
            cols = [c.strip() for c in pd.read_csv(f, nrows=0).columns]

            # Detect possible matching column names
            mapping = {}
            for c in cols:
                c_lower = c.lower()
                if "date" in c_lower or "time" in c_lower:
                    mapping[c] = "Timestamp"
                elif "pm2" in c_lower:
                    mapping[c] = "PM2.5"
                elif "pm10" in c_lower:
                    mapping[c] = "PM10"
                elif "o3" in c_lower or "ozone" in c_lower:
                    mapping[c] = "O3"
                elif "co" in c_lower and not "co2" in c_lower:
                    mapping[c] = "CO"

            if not mapping:
                print(f"⚠️ Skipped {f} (no relevant columns found)")
                continue

            df = pd.read_csv(f, usecols=mapping.keys(), encoding_errors="ignore")
            df.rename(columns=mapping, inplace=True)
            df["StationFile"] = os.path.basename(f)

            # Make sure all columns exist
            for col in ["Timestamp", "PM2.5", "PM10", "O3", "CO"]:
                if col not in df.columns:
                    df[col] = None

            if first:
                df.to_csv(out_file, index=False)
                first = False
            else:
                df.to_csv(out_file, mode="a", index=False, header=False)

            print(f"[{i}/{len(files)}] Processed {os.path.basename(f)} ({len(df)} rows)")
            del df

        except Exception as e:
            print(f"❌ Skipped {f}: {e}")

    print(f"\n✅ Done! Fixed merged file saved to: {out_file}")

# Run this to rebuild Kaggle merged dataset
load_pm_pollutants_fixed("data/kaggle_csvs", "data/kaggle_pm_merged_fixed.csv")

Found 454 CSV files.
[1/454] Processed AP001.csv (59150 rows)
[2/454] Processed AP002.csv (51864 rows)
[3/454] Processed AP003.csv (50400 rows)
[4/454] Processed AP004.csv (48802 rows)
[5/454] Processed AP005.csv (46880 rows)
[6/454] Processed AP006.csv (5432 rows)
[7/454] Processed AP007.csv (3127 rows)
[8/454] Processed AP008.csv (3280 rows)
[9/454] Processed AP009.csv (1689 rows)
[10/454] Processed AP010.csv (1593 rows)
[11/454] Processed AR001.csv (17655 rows)
[12/454] Processed AS001.csv (35808 rows)
[13/454] Processed AS002.csv (19764 rows)
[14/454] Processed AS003.csv (4641 rows)
[15/454] Processed AS004.csv (4400 rows)
[16/454] Processed AS005.csv (3081 rows)
[17/454] Processed AS006.csv (2462 rows)
[18/454] Processed AS007.csv (2390 rows)
[19/454] Processed AS008.csv (1881 rows)
[20/454] Processed AS009.csv (744 rows)
[21/454] Processed BR001.csv (94224 rows)
[22/454] Processed BR002.csv (65736 rows)
[23/454] Processed BR003.csv (62088 rows)
[24/454] Processed BR004.csv (28639

KeyboardInterrupt: 

In [ ]:
merged_kaggle = pd.read_csv("data/kaggle_pm_merged_fixed.csv", on_bad_lines="skip", low_memory=False)
merged_kaggle.head(10)

,Timestamp,Timestamp.1,PM2.5,PM10,CO,O3,StationFile
0,2016-07-01 10:00:00,2016-07-01 11:00:00,10.67,39.00,0.48,14.5,AP001.csv
1,2016-07-01 11:00:00,2016-07-01 12:00:00,2.00,39.00,0.49,15.0,AP001.csv
2,2016-07-01 12:00:00,2016-07-01 13:00:00,NaN,NaN,NaN,NaN,AP001.csv
3,2016-07-01 13:00:00,2016-07-01 14:00:00,NaN,NaN,NaN,NaN,AP001.csv
4,2016-07-01 14:00:00,2016-07-01 15:00:00,20.50,50.00,0.47,10.5,AP001.csv
5,2016-07-01 15:00:00,2016-07-01 16:00:00,15.25,59.50,0.51,6.6,AP001.csv
6,2016-07-01 16:00:00,2016-07-01 17:00:00,11.67,60.00,0.46,17.43,AP001.csv
7,2016-07-01 17:00:00,2016-07-01 18:00:00,11.75,57.50,0.44,19.98,AP001.csv
8,2016-07-01 18:00:00,2016-07-01 19:00:00,18.00,57.75,0.45,12.2,AP001.csv
9,2016-07-01 19:00:00,2016-07-01 20:00:00,12.00,63.00,0.41,17.5,AP001.csv


In [18]:
city_df = pd.read_csv("data/cities_combined.csv")

# Make sure Timestamp format matches
for df in [merged_kaggle, city_df]:
    for col in df.columns:
        if "time" in col.lower() or "date" in col.lower():
            df.rename(columns={col: "Timestamp"}, inplace=True)
            break
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

merged_kaggle["Source"] = "KaggleStation"
city_df["Source"] = "CityCombined"

keep_cols = ["Timestamp", "PM2.5", "PM10", "O3", "CO", "Source"]
for col in keep_cols:
    if col not in city_df.columns:
        city_df[col] = None

combined_df = pd.concat([merged_kaggle[keep_cols], city_df[keep_cols]], ignore_index=True)
print("✅ Combined dataset shape:", combined_df.shape)
combined_df.head()


✅ Combined dataset shape: (14313933, 6)


,Timestamp,PM2.5,PM10,O3,CO,Source
0,2016-07-01 10:00:00,10.67,39.0,14.5,0.48,KaggleStation
1,2016-07-01 11:00:00,2.00,39.0,15.0,0.49,KaggleStation
2,2016-07-01 12:00:00,NaN,NaN,NaN,NaN,KaggleStation
3,2016-07-01 13:00:00,NaN,NaN,NaN,NaN,KaggleStation
4,2016-07-01 14:00:00,20.50,50.0,10.5,0.47,KaggleStation


In [2]:
import pandas as pd

# Load the new Kaggle dataset (the fixed one)
kaggle_df = pd.read_csv("data/kaggle_pm_merged_fixed.csv", on_bad_lines="skip", low_memory=False)

# Load your Combined City Pollution dataset
city_df = pd.read_csv("data/cities_combined.csv")

print("Kaggle dataset shape:", kaggle_df.shape)
print("City dataset shape:", city_df.shape)

# Show first few rows from both
display(kaggle_df.head(3))
display(city_df.head(3))


Kaggle dataset shape: (907914, 7)
City dataset shape: (18270, 10)


,Timestamp,Timestamp.1,PM2.5,PM10,CO,O3,StationFile
0,2016-07-01 10:00:00,2016-07-01 11:00:00,10.67,39.0,0.48,14.5,AP001.csv
1,2016-07-01 11:00:00,2016-07-01 12:00:00,2.00,39.0,0.49,15.0,AP001.csv
2,2016-07-01 12:00:00,2016-07-01 13:00:00,NaN,NaN,NaN,NaN,AP001.csv


,Timestamp,Location,PM2.5,PM10,NO2,NH3,SO2,CO,O3,City
0,01-01-2020,Bengaluru - Silk Board,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bengaluru
1,02-01-2020,Bengaluru - Silk Board,43.67,134.00,20.28,10.98,3.41,0.91,21.82,Bengaluru
2,03-01-2020,Bengaluru - Silk Board,30.58,74.42,15.17,12.10,3.27,0.96,23.31,Bengaluru


In [3]:
# Normalize timestamp names
for df in [kaggle_df, city_df]:
    for col in df.columns:
        if "time" in col.lower() or "date" in col.lower():
            df.rename(columns={col: "Timestamp"}, inplace=True)
            break

# Force datetime conversion + DROP invalid timestamps
for df in [kaggle_df, city_df]:
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
    df.dropna(subset=["Timestamp"], inplace=True)

# Add source column
kaggle_df["Source"] = "KaggleStation"
city_df["Source"] = "CityCombined"

# Keep only the matching pollutant columns
keep_cols = ["Timestamp", "PM2.5", "PM10", "O3", "CO", "Source"]

# Add missing columns
for col in keep_cols:
    if col not in city_df.columns:
        city_df[col] = None

# Merge both datasets
combined_df = pd.concat(
    [kaggle_df[keep_cols], city_df[keep_cols]],
    ignore_index=True
)

print("✅ Combined dataset shape:", combined_df.shape)
combined_df.head()


✅ Combined dataset shape: (915114, 6)


,Timestamp,PM2.5,PM10,O3,CO,Source
0,2016-07-01 10:00:00,10.67,39.0,14.5,0.48,KaggleStation
1,2016-07-01 11:00:00,2.00,39.0,15.0,0.49,KaggleStation
2,2016-07-01 12:00:00,NaN,NaN,NaN,NaN,KaggleStation
3,2016-07-01 13:00:00,NaN,NaN,NaN,NaN,KaggleStation
4,2016-07-01 14:00:00,20.50,50.0,10.5,0.47,KaggleStation


In [ ]:
# Check missing values
print(combined_df.isna().sum())

# Drop rows with completely empty pollutant data
combined_df = combined_df.dropna(subset=["PM2.5", "PM10", "O3", "CO"], how="all")

# Fill remaining missing values with median (optional)
combined_df[["PM2.5", "PM10", "O3", "CO"]] = combined_df[["PM2.5", "PM10", "O3", "CO"]].fillna(
    combined_df.median(numeric_only=True)
)

print("✅ After cleaning:", combined_df.shape)
combined_df.head()

Timestamp      11070
PM2.5        3829962
PM10         4423580
O3           3294769
CO           3267773
Source             0
dtype: int64
✅ After cleaning: (12144881, 6)


,Timestamp,PM2.5,PM10,O3,CO,Source
0,2016-07-01 10:00:00,10.67,39.0,14.5,0.48,KaggleStation
1,2016-07-01 11:00:00,2.00,39.0,15.0,0.49,KaggleStation
4,2016-07-01 14:00:00,20.50,50.0,10.5,0.47,KaggleStation
5,2016-07-01 15:00:00,15.25,59.5,6.6,0.51,KaggleStation
6,2016-07-01 16:00:00,11.67,60.0,17.43,0.46,KaggleStation


In [ ]:
combined_df.to_csv("data/master_airquality_clean.csv", index=False)
print("✅ Saved clean dataset: data/master_airquality_clean.csv")

✅ Saved clean dataset: data/master_airquality_clean.csv


#NOTHING IS IMPORTANT AFTER THIS

In [ ]:
# pip install (run in a notebook cell with `!` or in terminal)
!pip install --upgrade pip
!pip install pandas numpy scikit-learn xgboost joblib matplotlib seaborn
!pip install aif360 metaflow wandb bentoml whylabs-sdk fastparquet

ERROR: To modify pip, please run the following command:
D:\anaconda3\python.exe -m pip install --upgrade pip



  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Using cached metaflow-2.19.7-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached wandb-0.23.0-py3-none-win_amd64.whl.metadata (12 kB)
  Using cached bentoml-1.4.28-py3-none-any.whl.metadata (16 kB)


ERROR: Could not find a version that satisfies the requirement whylabs-sdk (from versions: none)
ERROR: No matching distribution found for whylabs-sdk


In [ ]:
# Fix Timestamp -> datetime, create time features, and show diagnostics
import pandas as pd
import numpy as np

# load dataset if not in memory
try:
    df  # if df exists, we use it
except NameError:
    df = pd.read_csv("data/master_airquality_clean.csv", low_memory=False)

# Find likely timestamp column(s)
possible_ts = [c for c in df.columns if any(k in c.lower() for k in ["time","date","timestamp"])]
print("Timestamp-like columns found:", possible_ts)

# If 'Timestamp' already present but not datetime, try to convert it.
if 'Timestamp' in df.columns:
    ts_col = 'Timestamp'
else:
    ts_col = possible_ts[0] if possible_ts else None

if ts_col is None:
    raise RuntimeError("No timestamp-like column found. Please tell me the column names: " + ", ".join(df.columns))

# Convert robustly (try several formats)
df[ts_col] = pd.to_datetime(df[ts_col], errors='coerce', dayfirst=True, infer_datetime_format=True)

# If too many NaT, try alternate parsing (common alternative formats)
nat_frac = df[ts_col].isna().mean()
print(f"After first parse, NaT fraction = {nat_frac:.3f}")

if nat_frac > 0.25:
    # try parsing with no dayfirst
    df[ts_col] = pd.to_datetime(df[ts_col].astype(str), errors='coerce', dayfirst=False, infer_datetime_format=True)
    nat_frac2 = df[ts_col].isna().mean()
    print(f"After second parse (dayfirst=False), NaT fraction = {nat_frac2:.3f}")

# Rename unified column to 'Timestamp'
df.rename(columns={ts_col: 'Timestamp'}, inplace=True)

# Drop rows with missing Timestamp (can't use them for time features)
n_before = len(df)
df = df[~df['Timestamp'].isna()].copy()
n_after = len(df)
print(f"Dropped {n_before - n_after} rows with unparseable Timestamp")

# Create time features
df['hour'] = df['Timestamp'].dt.hour
df['dayofweek'] = df['Timestamp'].dt.dayofweek
df['month'] = df['Timestamp'].dt.month

# Ensure numeric pollutant columns exist
for col in ['PM2.5','PM10','O3','CO']:
    if col not in df.columns:
        df[col] = np.nan
    else:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Quick diagnostics
print("Dataset now has shape:", df.shape)
print("Timestamp min/max:", df['Timestamp'].min(), "/", df['Timestamp'].max())
display(df.head(5))

Timestamp-like columns found: ['Timestamp']


C:\Users\Vineet Raj\AppData\Local\Temp\ipykernel_23544\2278563494.py:25: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[ts_col] = pd.to_datetime(df[ts_col], errors='coerce', dayfirst=True, infer_datetime_format=True)


After first parse, NaT fraction = 0.608


C:\Users\Vineet Raj\AppData\Local\Temp\ipykernel_23544\2278563494.py:33: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[ts_col] = pd.to_datetime(df[ts_col].astype(str), errors='coerce', dayfirst=False, infer_datetime_format=True)


After second parse (dayfirst=False), NaT fraction = 0.608
Dropped 7378375 rows with unparseable Timestamp
Dataset now has shape: (4766506, 9)
Timestamp min/max: 2010-01-01 00:00:00 / 2024-12-12 00:00:00


,Timestamp,PM2.5,PM10,O3,CO,Source,hour,dayofweek,month
0,2016-01-07 10:00:00,10.67,39.0,14.50,0.48,KaggleStation,10,3,1
1,2016-01-07 11:00:00,2.00,39.0,15.00,0.49,KaggleStation,11,3,1
2,2016-01-07 14:00:00,20.50,50.0,10.50,0.47,KaggleStation,14,3,1
3,2016-01-07 15:00:00,15.25,59.5,6.60,0.51,KaggleStation,15,3,1
4,2016-01-07 16:00:00,11.67,60.0,17.43,0.46,KaggleStation,16,3,1


In [4]:
# Make sure features list exists
features = ['PM10', 'O3', 'CO', 'hour', 'dayofweek', 'month']

In [ ]:
from sklearn.model_selection import train_test_split

n = len(df)
test_size = int(0.2 * n)
train_df = df.iloc[:n - test_size].copy()
test_df  = df.iloc[n - test_size:].copy()

X_train = train_df[features]
y_train = train_df['PM2.5']
X_test  = test_df[features]
y_test  = test_df['PM2.5']

print("Train:", X_train.shape, "Test:", X_test.shape)

Train: (3813205, 6) Test: (953301, 6)


In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

# Impute (fill) missing values in features
imputer = SimpleImputer(strategy="median")
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=features)
X_test = pd.DataFrame(imputer.transform(X_test), columns=features)

# Just to confirm:
print("✅ Missing values after imputation:")
print("Train:", np.isnan(X_train.values).sum(), "Test:", np.isnan(X_test.values).sum())

✅ Missing values after imputation:
Train: 0 Test: 0


In [ ]:
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import os

os.makedirs("models", exist_ok=True)

def eval_model(name, model, X_test, y_test):
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)
    r2 = r2_score(y_test, preds)
    print(f"{name} -> RMSE: {rmse:.4f}, R2: {r2:.4f}")
    return {"rmse": rmse, "r2": r2}

results = {}

# Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
results['linear'] = eval_model("LinearRegression", lr, X_test, y_test)
joblib.dump(lr, "models/linear_reg.joblib")

# Random Forest
from sklearn.ensemble import RandomForestRegressor
import gc

rf = RandomForestRegressor(
    n_estimators=100,       # fewer trees (cut memory ~½)
    max_depth=20,           # limit depth to control tree size
    n_jobs=-1,              # still use all cores
    random_state=42
)
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)
results['rf'] = eval_model("RandomForest", rf, X_test, y_test)
joblib.dump(rf, "models/rf_reg.joblib")

gc.collect()   # free memory

results

d:\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


LinearRegression -> RMSE: 40.1835, R2: 0.6015


d:\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RandomForest -> RMSE: 39.7948, R2: 0.6091


d:\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


XGBoost -> RMSE: 40.0060, R2: 0.6050


{'linear': {'rmse': 40.183530025414086, 'r2': 0.6014529977703444},
 'rf': {'rmse': 39.794836109028864, 'r2': 0.6091259703523371},
 'xgb': {'rmse': 40.00601876637627, 'r2': 0.604966393340345}}

In [37]:
import xgboost as xgb
# XGBoost
xgr = xgb.XGBRegressor(n_estimators=300, tree_method='hist', random_state=42, verbosity=0)
xgr.fit(X_train, y_train)
results['xgb'] = eval_model("XGBoost", xgr, X_test, y_test)
xgr.save_model("models/xgb_reg.json")

results

d:\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


XGBoost -> RMSE: 40.0060, R2: 0.6050


{'linear': {'rmse': 40.183530025414086, 'r2': 0.6014529977703444},
 'rf': {'rmse': 39.794836109028864, 'r2': 0.6091259703523371},
 'xgb': {'rmse': 40.00601876637627, 'r2': 0.604966393340345}}

In [1]:
pip install protobuf==3.20.*

  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-sdk 0.73.0 requires protobuf!=5.26.*,!=5.27.*,!=5.28.*,!=5.29.0,!=5.29.1,!=5.29.2,!=5.29.3,!=5.29.4,!=6.30.0,!=6.30.1,!=6.31.0,<7.0,>=4.25.8, but you have protobuf 3.20.3 which is incompatible.
opentelemetry-proto 1.38.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.


In [6]:
# governance_check.py (run in notebook cell or save as file and run)
import pandas as pd
import numpy as np, json
from sklearn.metrics import mean_absolute_error
import joblib

# load data and model
df_all = df.copy()   # your cleaned dataframe in memory
model = joblib.load("models/rf_reg.joblib")  # or use best model

# Regression fairness: MAE by Source and (if available) by City
df_all['pred'] = model.predict(df_all[features].fillna(df_all[features].median()))
reg_mae_source = df_all.groupby('Source').apply(lambda g: mean_absolute_error(g['PM2.5'], g['pred'])).to_dict()
reg_mae_city = df_all.groupby('City').apply(lambda g: mean_absolute_error(g['PM2.5'], g['pred'])) if 'City' in df_all.columns else None

report = {
    "regression_mae_by_source": reg_mae_source,
    "sample_top_cities_mae": (reg_mae_city.sort_values(ascending=False).head(10).to_dict() if reg_mae_city is not None else {})
}

# AIF360 classification proxy (High vs NotHigh) + reweighing
try:
    from aif360.datasets import BinaryLabelDataset
    from aif360.algorithms.preprocessing import Reweighing

    df_clf = df_all.copy()
    df_clf['label_high'] = (df_clf['PM2.5'] >= 60).astype(int)
    df_clf['protected'] = (df_clf['Source'] == 'CityCombined').astype(int)
    X = df_clf[features].fillna(df_clf[features].median())
    data_for_aif = pd.DataFrame(np.hstack([X.values, df_clf['label_high'].values.reshape(-1,1), df_clf['protected'].values.reshape(-1,1)]),
                                columns = [*features,'label','protected'])
    dataset = BinaryLabelDataset(df=data_for_aif, label_names=['label'], protected_attribute_names=['protected'])
    rw = Reweighing(unprivileged_groups=[{'protected':0}], privileged_groups=[{'protected':1}])
    dataset_transf = rw.fit_transform(dataset)
    # show weight summary
    unique_weights = np.unique(dataset_transf.instance_weights)[:10].tolist()
    report['aif360_weights_sample'] = unique_weights
except Exception as e:
    report['aif360_error'] = str(e)

open("governance_report.json","w").write(json.dumps(report, indent=2))
print("Saved governance_report.json")
print(report)


C:\Users\Vineet Raj\AppData\Local\Temp\ipykernel_23544\3259342412.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reg_mae_source = df_all.groupby('Source').apply(lambda g: mean_absolute_error(g['PM2.5'], g['pred'])).to_dict()
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


Saved governance_report.json
{'regression_mae_by_source': {'CityCombined': 16.681804290273604, 'KaggleStation': 21.94193433674848}, 'sample_top_cities_mae': {}, 'aif360_weights_sample': [0.9393110488990309, 0.9999344632685842, 1.0000320881265077, 1.1520370612025757]}
